# Pre Process - Cleaning and Merging the Education Data

**Table of Contents**
1. [Intro](#1.-Intro)
2. [Cleaning Data](#2.-Cleaning_Data)
3. [Merging Data](#3.-Merging_Data)
4. [Handling Missing Data](#3.-Handling_Missing_Data)

## 1. Intro
After getting the data from different endpoints, we merged tables and cleaned data, including complementing the missing data.

In [14]:
import pandas as pd, numpy as np

In [15]:
# Load data from different endpoints
school_dir_2013 = pd.read_csv('./educationdata/ccd_directory_2013.csv')
school_dir_2015 = pd.read_csv('./educationdata/ccd_directory_2015.csv')
school_dir_2017 = pd.read_csv('./educationdata/ccd_directory_2017.csv')

g9_enrollment_2013 = pd.read_csv('./educationdata/ccd_g9_enrollment_2013.csv')
g9_enrollment_2015 = pd.read_csv('./educationdata/ccd_g9_enrollment_2015.csv')
g9_enrollment_2017 = pd.read_csv('./educationdata/ccd_g9_enrollment_2017.csv')
g10_enrollment_2013 = pd.read_csv('./educationdata/ccd_g10_enrollment_2013.csv')
g10_enrollment_2015 = pd.read_csv('./educationdata/ccd_g10_enrollment_2015.csv')
g10_enrollment_2017 = pd.read_csv('./educationdata/ccd_g10_enrollment_2017.csv')
g11_enrollment_2013 = pd.read_csv('./educationdata/ccd_g11_enrollment_2013.csv')
g11_enrollment_2015 = pd.read_csv('./educationdata/ccd_g11_enrollment_2015.csv')
g11_enrollment_2017 = pd.read_csv('./educationdata/ccd_g11_enrollment_2017.csv')
g12_enrollment_2013 = pd.read_csv('./educationdata/ccd_enrollment_2013.csv')
g12_enrollment_2015 = pd.read_csv('./educationdata/ccd_enrollment_2015.csv')
g12_enrollment_2017 = pd.read_csv('./educationdata/ccd_enrollment_2017.csv')

g12_retention_2013 = pd.read_csv('./educationdata/crdc_retention_2013.csv')
g12_retention_2015 = pd.read_csv('./educationdata/crdc_retention_2015.csv')
g12_retention_2017 = pd.read_csv('./educationdata/crdc_retention_2017.csv')

sat_act_participation_2013 = pd.read_csv('./educationdata/crdc_sat_act_part_2013.csv')
sat_act_participation_2015 = pd.read_csv('./educationdata/crdc_sat_act_part_2015.csv')
sat_act_participation_2017 = pd.read_csv('./educationdata/crdc_sat_act_part_2017.csv')

finance_2013 = pd.read_csv('./educationdata/crdc_school_finance_2013.csv')
finance_2015 = pd.read_csv('./educationdata/crdc_school_finance_2015.csv')
finance_2017 = pd.read_csv('./educationdata/crdc_school_finance_2017.csv')

C:\Users\ymsc1\anaconda3\envs\Data Science\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ymsc1\anaconda3\envs\Data Science\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (47,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ymsc1\anaconda3\envs\Data Science\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ymsc1\anaconda3\envs\Data Science\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (2,5) have mixed types.Specify dtype option on import or set low_memory=Fal

## 2. Cleaning Data
According to the data source, we are making functions to deal with different conditions. It generally follows these steps:
1. Drop the unnecessary columns
2. Uniform the ncessch column to 12-digit string as the key for merging the dataframes
3. Fill missing data with N/A
4. Drop duplicated data
5. Drop some parts with too much missing data

In [16]:
def clean_school_dir(df):
    # Drop the columns with nearly total missing values
    df.drop(['Unnamed: 0', 'csa', 'cbsa', 'virtual', 'state_leg_district_lower', 'state_leg_district_upper', \
             'congress_district_id', 'direct_certification', 'lunch_program', 'ncessch_num', 'bureau_indian_education', \
             'title_i_status', 'title_i_eligible', 'title_i_schoolwide', 'magnet', 'shared_time', \
             'free_lunch', 'reduced_price_lunch', 'free_or_reduced_price_lunch'], axis = 1, inplace = True)
    # Uniforming the ncessch column to 12-digit string as the key for merging the dataframes
    df.ncessch = df.ncessch.astype(str).str.zfill(12)
    df.zip_location = df.zip_location.astype(str)
    df.county_code = df.county_code.astype(str).str.zfill(5)
    # Filling in missing data with nan 
    df.state_location.replace(to_replace = '-1', value = np.nan, inplace =True) 
    df.state_location.fillna(df.state_mailing, inplace = True)
    df.zip_location.replace(to_replace = -1, value = np.nan, inplace =True) 
    df.zip_location.fillna(df.zip_mailing, inplace = True)
    # Drop states which have too much missing data
    df.drop(df[df['state_location'] == 'AE'].index, inplace = True)
    df.drop(df[df['state_location'] == 'AP'].index, inplace = True)
    df.drop(df[df['state_location'] == 'AS'].index, inplace = True)
    df.drop(df[df['state_location'] == 'GU'].index, inplace = True)
    df.drop(df[df['state_location'] == 'PR'].index, inplace = True)
    df.drop(df[df['state_location'] == 'VI'].index, inplace = True)
    return df

school_dir_2013 = clean_school_dir(school_dir_2013)
school_dir_2015 = clean_school_dir(school_dir_2015)
school_dir_2017 = clean_school_dir(school_dir_2017)

def clean_enrollment(df):
    # Select columns as needed
    df = df[['ncessch', 'enrollment']]
    # Uniforming the ncessch column
    df.ncessch = df.ncessch.astype(str).str.zfill(12)
    # Drop duplicated data
    df.drop_duplicates(subset=['ncessch'], keep = 'first', inplace = True)
    # Filling in missing data with nan
    df.enrollment.replace(to_replace = -2, value = 0, inplace =True) 
    df.enrollment.replace(to_replace = -3, value = 0, inplace =True) 
    return df
g9_enrollment_2013 = clean_enrollment(g9_enrollment_2013).rename(columns = {'enrollment':'g9_enrollment'})
g9_enrollment_2015 = clean_enrollment(g9_enrollment_2015).rename(columns = {'enrollment':'g9_enrollment'})
g9_enrollment_2017 = clean_enrollment(g9_enrollment_2017).rename(columns = {'enrollment':'g9_enrollment'})
g10_enrollment_2013 = clean_enrollment(g10_enrollment_2013).rename(columns = {'enrollment':'g10_enrollment'})
g10_enrollment_2015 = clean_enrollment(g10_enrollment_2015).rename(columns = {'enrollment':'g10_enrollment'})
g10_enrollment_2017 = clean_enrollment(g10_enrollment_2017).rename(columns = {'enrollment':'g10_enrollment'})
g11_enrollment_2013 = clean_enrollment(g11_enrollment_2013).rename(columns = {'enrollment':'g11_enrollment'})
g11_enrollment_2015 = clean_enrollment(g11_enrollment_2015).rename(columns = {'enrollment':'g11_enrollment'})
g11_enrollment_2017 = clean_enrollment(g11_enrollment_2017).rename(columns = {'enrollment':'g11_enrollment'})
g12_enrollment_2013 = clean_enrollment(g12_enrollment_2013).rename(columns = {'enrollment':'g12_enrollment'})
g12_enrollment_2015 = clean_enrollment(g12_enrollment_2015).rename(columns = {'enrollment':'g12_enrollment'})
g12_enrollment_2017 = clean_enrollment(g12_enrollment_2017).rename(columns = {'enrollment':'g12_enrollment'})

def clean_retention(df):
    df = df[['ncessch', 'students_retained']]
    df.ncessch = df.ncessch.astype(str).replace('\.0', '', regex=True).str.zfill(12)
    df.drop_duplicates(subset=['ncessch'], keep = 'first', inplace = True)
    df.fillna(0, inplace = True)
    return df

g12_retention_2013 = clean_retention(g12_retention_2013)
g12_retention_2015 = clean_retention(g12_retention_2015)
g12_retention_2017 = clean_retention(g12_retention_2017)

def clean_sat_act(df):
    df = df[['ncessch', 'students_SAT_ACT']]
    df.ncessch = df.ncessch.astype(str).replace('\.0', '', regex=True).str.zfill(12)
    df.drop_duplicates(subset=['ncessch'], keep = 'first', inplace = True)
    return df

sat_act_participation_2013 = clean_sat_act(sat_act_participation_2013)
sat_act_participation_2015 = clean_sat_act(sat_act_participation_2015)
sat_act_participation_2017 = clean_sat_act(sat_act_participation_2017)

def clean_finance(df):
    df = df[['ncessch', 'salaries_teachers', 'salaries_total', 'expenditures_nonpersonnel']]
    df.ncessch = df.ncessch.astype(str).replace('\.0', '', regex=True).str.zfill(12)
    df.drop_duplicates(subset=['ncessch'], keep = 'first', inplace = True)
    return df

finance_2013 = clean_finance(finance_2013)
finance_2015 = clean_finance(finance_2015)
finance_2017 = clean_finance(finance_2017)

C:\Users\ymsc1\anaconda3\envs\Data Science\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\ymsc1\AppData\Local\Temp/ipykernel_9488/1997949834.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['ncessch'], keep = 'first', inplace = True)
C:\Users\ymsc1\anaconda3\envs\Data Science\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pan

## 3. Merging Data
Merge all the processed tables using left-join with 'ncessch' as the key. 

In [17]:
# merge the dataframes
def merge_school_data(df_dir, df_erg9, df_erg10, df_erg11, df_erg12, df_rtn, df_sat, df_fnc):
    df_dir = df_dir.merge(df_erg9, on = 'ncessch', how = 'left')
    df_dir = df_dir.merge(df_erg10, on = 'ncessch', how = 'left')
    df_dir = df_dir.merge(df_erg11, on = 'ncessch', how = 'left')
    df_dir = df_dir.merge(df_erg12, on = 'ncessch', how = 'left')
    df_dir = df_dir.merge(df_rtn, on = 'ncessch', how = 'left')
    df_dir = df_dir.merge(df_sat, on = 'ncessch', how = 'left')
    df_dir = df_dir.merge(df_fnc, on = 'ncessch', how = 'left')
    df_dir = df_dir.rename(columns = {'students_retained':'g12_students_retained'})
# According to crdc document, missing values are encoded as -1 which should be converted to nan.
    df_dir.replace(to_replace = -1, value = np.nan, inplace =True) 
    return df_dir

school_dir_2013 = merge_school_data(school_dir_2013, g9_enrollment_2013, g10_enrollment_2013, g11_enrollment_2013, g12_enrollment_2013, g12_retention_2013, sat_act_participation_2013, finance_2013)
school_dir_2015 = merge_school_data(school_dir_2015, g9_enrollment_2015, g10_enrollment_2015, g11_enrollment_2015, g12_enrollment_2015, g12_retention_2015, sat_act_participation_2015, finance_2015)
school_dir_2017 = merge_school_data(school_dir_2017, g9_enrollment_2017, g10_enrollment_2017, g11_enrollment_2017, g12_enrollment_2017, g12_retention_2017, sat_act_participation_2017, finance_2017)


## 4. Handling Missing Data
We defined some rules as below for filling the missing values:
1. Filling the missing enrollment value by another grade in the same school
2. Updating the total enrollment value by the sum of G9, G10, G11, and G12
3. Filling the missing value of teachers_fte and finance data by using the state means

In [18]:
# g12_enrollment g12_students_retained students_SAT_ACT salaries_teachers salaries_total expenditures_nonpersonnel
def fill_missing_value(df, filename):
    # Filling the missing enrollment value by another grade in the same school
    df.g12_enrollment = df.g12_enrollment.fillna(df.g11_enrollment)
    df.g11_enrollment = df.g11_enrollment.fillna(df.g10_enrollment)
    df.g10_enrollment = df.g10_enrollment.fillna(df.g9_enrollment)
    df.g9_enrollment = df.g9_enrollment.fillna(df.g10_enrollment)
    # Updating the total enrollment value by the sum of G9, G10, G11, and G12
    df.enrollment = df.g9_enrollment + df.g10_enrollment + df.g11_enrollment + df.g12_enrollment
    # Filling the missing value of teachers_fte and finance data by using the state means
    df.teachers_fte = df.teachers_fte.fillna(df.groupby('state_location').teachers_fte.transform('mean'))
    df.salaries_teachers = df.salaries_teachers.fillna(df.groupby('state_location').salaries_teachers.transform('mean'))
    df.salaries_total = df.salaries_total.fillna(df.groupby('state_location').salaries_total.transform('mean'))
    df.expenditures_nonpersonnel = df.expenditures_nonpersonnel.fillna(df.groupby('state_location').expenditures_nonpersonnel.transform('mean'))
    df.fillna(0, inplace = True)
    # Save dataframes to csv files
    df.to_csv("./educationdata/" + filename + ".csv")
    return df
school_dir_2013 = fill_missing_value(school_dir_2013, 'school_dir_2013')
school_dir_2015 = fill_missing_value(school_dir_2015, 'school_dir_2015')
school_dir_2017 = fill_missing_value(school_dir_2017,'school_dir_2017')

### Inspection for final check...

In [19]:
school_dir_2013.isnull().sum()

year                         0
ncessch                      0
school_id                    0
school_name                  0
leaid                        0
lea_name                     0
state_leaid                  0
seasch                       0
street_mailing               0
city_mailing                 0
state_mailing                0
zip_mailing                  0
street_location              0
city_location                0
state_location               0
zip_location                 0
phone                        0
fips                         0
latitude                     0
longitude                    0
urban_centric_locale         0
county_code                  0
school_level                 0
school_type                  0
school_status                0
lowest_grade_offered         0
highest_grade_offered        0
charter                      0
teachers_fte                 0
elem_cedp                    0
high_cedp                    0
middle_cedp                  0
ungrade_

In [20]:
school_dir_2015.isnull().sum()

year                         0
ncessch                      0
school_id                    0
school_name                  0
leaid                        0
lea_name                     0
state_leaid                  0
seasch                       0
street_mailing               0
city_mailing                 0
state_mailing                0
zip_mailing                  0
street_location              0
city_location                0
state_location               0
zip_location                 0
phone                        0
fips                         0
latitude                     0
longitude                    0
urban_centric_locale         0
county_code                  0
school_level                 0
school_type                  0
school_status                0
lowest_grade_offered         0
highest_grade_offered        0
charter                      0
teachers_fte                 0
elem_cedp                    0
high_cedp                    0
middle_cedp                  0
ungrade_

In [21]:
school_dir_2017.isnull().sum()

year                         0
ncessch                      0
school_id                    0
school_name                  0
leaid                        0
lea_name                     0
state_leaid                  0
seasch                       0
street_mailing               0
city_mailing                 0
state_mailing                0
zip_mailing                  0
street_location              0
city_location                0
state_location               0
zip_location                 0
phone                        0
fips                         0
latitude                     0
longitude                    0
urban_centric_locale         0
county_code                  0
school_level                 0
school_type                  0
school_status                0
lowest_grade_offered         0
highest_grade_offered        0
charter                      0
teachers_fte                 0
elem_cedp                    0
high_cedp                    0
middle_cedp                  0
ungrade_

### Inspection from aggregation views...

In [22]:
def create_state_data(df, filename):
# groupby state and calculate aggregated values
    df = df.groupby('state_location')
    school_stats = df.size().to_frame(name = 'school_counts')
    school_stats = (school_stats
    .join(df.agg({'enrollment':'mean'}).round(0).rename(columns = {'enrollment':'enrollment_mean'}))
    .join(df.agg({'g9_enrollment':'mean'}).round(0).rename(columns = {'g9_enrollment':'g9_enrollment_mean'}))
    .join(df.agg({'g10_enrollment':'mean'}).round(0).rename(columns = {'g10_enrollment':'g10_enrollment_mean'}))
    .join(df.agg({'g11_enrollment':'mean'}).round(0).rename(columns = {'g11_enrollment':'g11_enrollment_mean'}))
    .join(df.agg({'g12_enrollment':'mean'}).round(0).rename(columns = {'g12_enrollment':'g12_enrollment_mean'}))
    .join(df.agg({'teachers_fte':'mean'}).round(0).rename(columns = {'teachers_fte':'teachers_fte_mean'}))
    .join(df.agg({'g12_students_retained':'mean'}).round(0).rename(columns = {'g12_students_retained':'g12_students_retained_mean'}))
    .join(df.agg({'students_SAT_ACT':'mean'}).round(0).rename(columns = {'students_SAT_ACT':'students_SAT_ACT_mean'}))
    .join(df.agg({'salaries_teachers':'mean'}).round(2).rename(columns = {'salaries_teachers':'salaries_teachers_mean'}))
    .join(df.agg({'salaries_total':'mean'}).round(2).rename(columns = {'salaries_total':'salaries_total_mean'}))
    .join(df.agg({'expenditures_nonpersonnel':'mean'}).round(2).rename(columns = {'expenditures_nonpersonnel':'expenditures_nonpersonnel_mean'}))
    ).reset_index()
# Uncomment to save dataframes to csv files
# school_stats.to_csv("./educationdata/" + filename + ".csv")
    return school_stats

state_data_2013 = create_state_data(school_dir_2013, 'state_data_2013')
state_data_2013.head()

,state_location,school_counts,enrollment_mean,g9_enrollment_mean,g10_enrollment_mean,g11_enrollment_mean,g12_enrollment_mean,teachers_fte_mean,g12_students_retained_mean,students_SAT_ACT_mean,salaries_teachers_mean,salaries_total_mean,expenditures_nonpersonnel_mean
0,AK,48,554.0,151.0,140.0,139.0,124.0,31.0,7.0,84.0,2054320.71,2914712.12,399878.79
1,AL,279,692.0,192.0,180.0,164.0,156.0,44.0,5.0,182.0,1993034.60,3550213.79,1081459.48
2,AR,259,500.0,132.0,130.0,123.0,114.0,39.0,1.0,129.0,1624452.08,2514754.25,781916.86
3,AZ,421,712.0,183.0,178.0,170.0,182.0,54.0,12.0,87.0,1498513.57,2248528.19,685308.58
4,CA,1253,1374.0,364.0,353.0,333.0,324.0,57.0,7.0,202.0,3803749.80,5793708.15,912939.49


In [23]:
def create_zip_data(df, filename):
# groupby state and calculate aggregated values
    df = df.groupby(['state_location', 'zip_location'])
    school_stats = df.size().to_frame(name = 'school_counts')
    school_stats = (school_stats
    .join(df.agg({'enrollment':'mean'}).round(0).rename(columns = {'enrollment':'enrollment_mean'}))
    .join(df.agg({'g9_enrollment':'mean'}).round(0).rename(columns = {'g9_enrollment':'g9_enrollment_mean'}))
    .join(df.agg({'g10_enrollment':'mean'}).round(0).rename(columns = {'g10_enrollment':'g10_enrollment_mean'}))
    .join(df.agg({'g11_enrollment':'mean'}).round(0).rename(columns = {'g11_enrollment':'g11_enrollment_mean'}))
    .join(df.agg({'g12_enrollment':'mean'}).round(0).rename(columns = {'g12_enrollment':'g12_enrollment_mean'}))
    .join(df.agg({'teachers_fte':'mean'}).round(0).rename(columns = {'teachers_fte':'teachers_fte_mean'}))
    .join(df.agg({'g12_students_retained':'mean'}).round(0).rename(columns = {'g12_students_retained':'g12_students_retained_mean'}))
    .join(df.agg({'students_SAT_ACT':'mean'}).round(0).rename(columns = {'students_SAT_ACT':'students_SAT_ACT_mean'}))
    .join(df.agg({'salaries_teachers':'mean'}).round(2).rename(columns = {'salaries_teachers':'salaries_teachers_mean'}))
    .join(df.agg({'salaries_total':'mean'}).round(2).rename(columns = {'salaries_total':'salaries_total_mean'}))
    .join(df.agg({'expenditures_nonpersonnel':'mean'}).round(2).rename(columns = {'expenditures_nonpersonnel':'expenditures_nonpersonnel_mean'}))
    ).reset_index()
    # Uncomment to save dataframes to csv files
    # school_stats.to_csv("./educationdata/" + filename + ".csv")
    return school_stats

zip_data_2013 = create_zip_data(school_dir_2013, 'zip_data_2013')
zip_data_2013.head()

,state_location,zip_location,school_counts,enrollment_mean,g9_enrollment_mean,g10_enrollment_mean,g11_enrollment_mean,g12_enrollment_mean,teachers_fte_mean,g12_students_retained_mean,students_SAT_ACT_mean,salaries_teachers_mean,salaries_total_mean,expenditures_nonpersonnel_mean
0,AK,99502,1,1647.0,425.0,433.0,408.0,381.0,80.0,21.0,325.0,4776774.0,6950663.37,171278.0
1,AK,99504,1,1612.0,456.0,411.0,372.0,373.0,81.0,30.0,196.0,4114336.0,6203010.90,196765.0
2,AK,99507,1,1748.0,474.0,436.0,428.0,410.0,81.0,32.0,268.0,4561540.0,6649633.59,177661.0
3,AK,99508,1,2139.0,593.0,567.0,492.0,487.0,107.0,40.0,269.0,5269608.0,7700904.21,215056.0
4,AK,99516,1,1448.0,387.0,368.0,371.0,322.0,70.0,17.0,341.0,4343927.0,6249713.91,183037.0
